**Mounting the Drive and other Imports**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00


In [ ]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1-cp310-cp310-linux_x86_64.whl size=661204 sha256=8a6cb642152dd282647953478614c153aa27e877a50fc0592a688e44e7b212ea
  Stored in directory: /root/.cache/pip/wheels/12/34/d2/9a349255a4eca3a486d82c79d21e138ce2ccd90f414d9d72b8
Successfully built pycuda


In [ ]:
from timeit import default_timer as timer
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np
import glob
import pandas as pd
import networkx as nx
import torch



```
# This is formatted as code
```

**CUDA kernel for computing distances between nodes and finding k nearest neighbors**

In [ ]:
distance_kernel = SourceModule("""
#include <math.h>

__device__ void selection_sort(float *data, int *indices, int left, int right, int num_next_nodes)
{
    for (int i = left; i <= right; ++i) {
        int min_idx = i;

        // Find the index of the smallest value in the range [left, right].
        for (int j = i + 1; j < num_next_nodes; ++j) {
            if (data[j] < data[min_idx]) {
                min_idx = j;
            }
        }

        // Swap the values in the data array and corresponding indices array.
        if (i != min_idx) {
            float temp = data[i];
            data[i] = data[min_idx];
            data[min_idx] = temp;

            int temp_idx = indices[i];
            indices[i] = indices[min_idx];
            indices[min_idx] = temp_idx;
        }
    }
}
__global__ void compute_distances(float *x_current, float *y_current, float *x_next, float *y_next, float *distances, int *indices, int num_current_nodes, int num_next_nodes) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i < num_current_nodes) {
        // Compute distances
        for (int j = 0; j < num_next_nodes; ++j) {
            distances[(i * num_next_nodes) + j] = sqrtf(powf(x_current[i] - x_next[j], 2) + powf(y_current[i] - y_next[j], 2));
        }

        // Initialize indices array
        for (int j = 0; j < num_next_nodes; ++j) {
            indices[(i * num_next_nodes) + j] = j;
        }

        // Selection sort to sort the distances and corresponding indices
        selection_sort(distances + i * num_next_nodes, indices + i * num_next_nodes, 0, num_next_nodes - 1, num_next_nodes);
    }
}

""")

# Get the kernel function
distance_func = distance_kernel.get_function("compute_distances")

def compute_distances_cuda(x_current, y_current, x_next, y_next, k=5):
    num_current_nodes = len(x_current)
    num_next_nodes = len(x_next)
    #allocating memory in gpu
    x_current_gpu = cuda.to_device(x_current)
    y_current_gpu = cuda.to_device(y_current)
    x_next_gpu = cuda.to_device(x_next)
    y_next_gpu = cuda.to_device(y_next)
    block_size = (256, 1, 1)  # Block dimensions: (threads_per_block_x, threads_per_block_y, threads_per_block_z)
    grid_size = ((num_current_nodes + block_size[0] - 1) // block_size[0], 1, 1)

    # Allocate memory for distances and nearest neighbor indices
    distances_gpu = cuda.mem_alloc(num_current_nodes * num_next_nodes * 4)  # Assuming float32
    indices_gpu = cuda.mem_alloc(num_current_nodes * num_next_nodes * 4)  # Assuming int32

    # Compute distances
    distance_func = distance_kernel.get_function("compute_distances")
    distance_func(x_current_gpu, y_current_gpu, x_next_gpu, y_next_gpu, distances_gpu, indices_gpu, np.int32(num_current_nodes), np.int32(num_next_nodes), block=block_size, grid=grid_size)

    # Copy distances and indices back to host
    distances_host = np.empty((num_current_nodes, num_next_nodes), dtype=np.float32)
    indices_host = np.empty((num_current_nodes, num_next_nodes), dtype=np.int32)
    cuda.memcpy_dtoh(distances_host, distances_gpu)
    cuda.memcpy_dtoh(indices_host, indices_gpu)


    # Free memory on GPU
    x_current_gpu.free()
    y_current_gpu.free()
    x_next_gpu.free()
    y_next_gpu.free()
    distances_gpu.free()
    indices_gpu.free()

    return distances_host, indices_host

# distance_kernel = SourceModule("""
# #include <math.h>

# __global__ void compute_distances(float *x_current, float *y_current, float *x_next, float *y_next, float *distances, int *indices, int num_current_nodes, int num_next_nodes) {
#     int i = blockIdx.x * blockDim.x + threadIdx.x;
#     if (i < num_current_nodes) {
#         // Compute distances
#         for (int j = 0; j < num_next_nodes; ++j) {
#             distances[(i * num_next_nodes) + j] = sqrtf(powf(x_current[i] - x_next[j], 2) + powf(y_current[i] - y_next[j], 2));
#             indices[(i * num_next_nodes) + j] = j;
#         }

#         // Selection sort to sort the distances and corresponding indices
#         for (int j = 0; j < num_next_nodes - 1; ++j) {
#             int min_idx = j;
#             for (int k = j + 1; k < num_next_nodes; ++k) {
#                 if (distances[(i * num_next_nodes) + k] < distances[(i * num_next_nodes) + min_idx]) {
#                     min_idx = k;
#                 }
#             }
#             // Swap distances
#             float temp_dist = distances[(i * num_next_nodes) + j];
#             distances[(i * num_next_nodes) + j] = distances[(i * num_next_nodes) + min_idx];
#             distances[(i * num_next_nodes) + min_idx] = temp_dist;

#             // Swap indices
#             int temp_idx = indices[(i * num_next_nodes) + j];
#             indices[(i * num_next_nodes) + j] = indices[(i * num_next_nodes) + min_idx];
#             indices[(i * num_next_nodes) + min_idx] = temp_idx;
#         }
#     }
# }
# """)

# def compute_distances_cuda(x_current, y_current, x_next, y_next):
#     num_current_nodes = len(x_current)
#     num_next_nodes = len(x_next)

#     # Allocate memory in GPU
#     x_current_gpu = cuda.to_device(x_current)
#     y_current_gpu = cuda.to_device(y_current)
#     x_next_gpu = cuda.to_device(x_next)
#     y_next_gpu = cuda.to_device(y_next)

#     block_size = (256, 1, 1)  # Block dimensions: (threads_per_block_x, threads_per_block_y, threads_per_block_z)
#     grid_size = ((num_current_nodes + block_size[0] - 1) // block_size[0], 1, 1)

#     # Allocate memory for distances and nearest neighbor indices
#     distances_gpu = cuda.mem_alloc(num_current_nodes * num_next_nodes * 4)  # Assuming float32
#     indices_gpu = cuda.mem_alloc(num_current_nodes * num_next_nodes * 4)  # Assuming int32

#     # Get the kernel function
#     distance_func = distance_kernel.get_function("compute_distances")

#     # Call the CUDA kernel
#     distance_func(x_current_gpu, y_current_gpu, x_next_gpu, y_next_gpu, distances_gpu, indices_gpu, np.int32(num_current_nodes), np.int32(num_next_nodes), block=block_size, grid=grid_size)

#     # Copy distances and indices back to host
#     distances_host = np.empty((num_current_nodes, num_next_nodes), dtype=np.float32)
#     indices_host = np.empty((num_current_nodes, num_next_nodes), dtype=np.int32)
#     cuda.memcpy_dtoh(distances_host, distances_gpu)
#     cuda.memcpy_dtoh(indices_host, indices_gpu)

#     # Free memory on GPU
#     x_current_gpu.free()
#     y_current_gpu.free()
#     x_next_gpu.free()
#     y_next_gpu.free()
#     distances_gpu.free()
#     indices_gpu.free()

#     return distances_host, indices_host










**CUDA kernel for feature matrix**



In [ ]:

def process_features_cuda(file_name):
    # Read CSV file and extract data
    adc_data = []
    with open(file_name, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            adc_data.append(row)

    # Convert data to numpy arrays
    num_nodes = len(adc_data)
    adc_values = np.array([float(row['Eff_ADC']) for row in adc_data], dtype=np.float32)
    layer_values = np.array([int(row['Layer']) for row in adc_data], dtype=np.int32)
    x_values = np.array([float(row['X']) for row in adc_data], dtype=np.float32)
    y_values = np.array([float(row['Y']) for row in adc_data], dtype=np.float32)

    #Allocating memory in gpu
    adc_values_gpu = cuda.mem_alloc(adc_values.nbytes)
    layer_values_gpu = cuda.mem_alloc(layer_values.nbytes)
    x_values_gpu = cuda.mem_alloc(x_values.nbytes)
    y_values_gpu = cuda.mem_alloc(y_values.nbytes)

    # Transfer data from host to device
    cuda.memcpy_htod(adc_values_gpu, adc_values)
    cuda.memcpy_htod(layer_values_gpu, layer_values)
    cuda.memcpy_htod(x_values_gpu, x_values)
    cuda.memcpy_htod(y_values_gpu, y_values)

    # Define CUDA kernel code
    cuda_kernel_code = """
    __global__ void processFeatures(float* adc_values, int* layer_values, float* x_values, float* y_values, int num_nodes, float* feature_matrix) {
        int tid = blockIdx.x * blockDim.x + threadIdx.x;
        if (tid < num_nodes) {
            // Process node features here
            float adc = adc_values[tid];
            int layer = layer_values[tid];
            float x = x_values[tid];
            float y = y_values[tid];
            // Example: Store features in the feature matrix
            int index = tid * 4; // Assuming each feature takes up 4 floats (ADC, Layer, X, Y)
            feature_matrix[index] = adc;
            feature_matrix[index + 1] = layer;
            feature_matrix[index + 2] = x;
            feature_matrix[index + 3] = y;
        }
    }
    """


    module = SourceModule(cuda_kernel_code)

    # kernel function
    kernel_function = module.get_function("processFeatures")

    feature_matrix_gpu = cuda.mem_alloc(num_nodes * 4 * np.float32().itemsize)

    block_dim = (256, 1, 1)
    grid_dim = ((num_nodes + block_dim[0] - 1) // block_dim[0], 1)
    feature_matrix = np.empty((num_nodes, 4), dtype=np.float32)

    kernel_function(adc_values_gpu, layer_values_gpu, x_values_gpu, y_values_gpu, np.int32(num_nodes), feature_matrix_gpu, block=block_dim, grid=grid_dim)
    cuda.memcpy_dtoh(feature_matrix, feature_matrix_gpu)

    adc_values_gpu.free()
    layer_values_gpu.free()
    x_values_gpu.free()
    y_values_gpu.free()
    feature_matrix_gpu.free()


    return feature_matrix



**Execution Part**

In [ ]:
import csv
file_names = glob.glob('/content/drive/MyDrive/tifr/*.csv')
x = []
edge_index = []
energy = []
start=timer()
for file_name in file_names:
        f = int(file_name.split('_')[-1].split('.')[0])

        print(f'Processing graph for electron {f}')
        df = pd.read_csv(file_name)

        if df.empty:
            print("Empty DataFrame for event", f)
            continue

        layer_data = []
        energy_event = df['E'].iloc[0]
        energy.append(energy_event)
        print("Energy for event", f, "is equal to:", energy_event)

        df.sort_values(['Layer', 'Eff_ADC'], inplace=True)

        graph = nx.Graph()

        for index, row in df.iterrows():
            graph.add_node(index, x=row['X'], y=row['Y'], layer=row['Layer'], adc=row['Eff_ADC'])

        layers = df['Layer'].unique()

        #connecting all nodes in a layer to a single node (here its node 0, it means in every layer, all nodes of each layer are connected to node 0 of that layer)
        for l in layers:
            nodes_in_layer = df[df['Layer'] == l].index.tolist()
            first_node = nodes_in_layer[0]
            for node in nodes_in_layer[1:]:
                graph.add_edge(first_node, node)

        #connecting nodes of consecutive layers...only nearest 5 nodes of the consecutive layer are connected to nodes of current layer
        for i in range(len(layers) - 1):
            current_layer_nodes = df[df['Layer'] == layers[i]].index.tolist()
            next_layer_nodes = df[df['Layer'] == layers[i+1]].index.tolist()
            x_current = np.array([df.loc[node, 'X'] for node in current_layer_nodes], dtype=np.float32)
            y_current = np.array([df.loc[node, 'Y'] for node in current_layer_nodes], dtype=np.float32)
            x_next = np.array([df.loc[node, 'X'] for node in next_layer_nodes], dtype=np.float32)
            y_next = np.array([df.loc[node, 'Y'] for node in next_layer_nodes], dtype=np.float32)

            # Find k nearest neighbors using CUDA
            sorted_distances, sorted_indices = compute_distances_cuda(x_current, y_current, x_next, y_next)
            print('Sorted distances for layer' + str(i))
            print(sorted_distances)
            for j, current_node in enumerate(current_layer_nodes):
              num_nearest_nodes = min(5, len(next_layer_nodes))  # Ensure we don't exceed the size of the next layer
              for k in range(num_nearest_nodes):  # Select the nearest nodes up to the size of the next layer
                  nearest_node = next_layer_nodes[sorted_indices[j][k]]  # Get node index from sorted indices
                  graph.add_edge(current_node, nearest_node)


        vertices = set()
        for edge in graph.edges():
            vertices.update(edge)

        # Create a dictionary to map each vertex to a unique index
        index_map = {vertex: i for i, vertex in enumerate(vertices)}

        # Create the adjacency matrix
        n = len(vertices)
        adj_matrix = [[0] * n for _ in range(n)]
        for edge in graph.edges():
            i, j = index_map[edge[0]], index_map[edge[1]]
            distance = ((graph.nodes[edge[0]]['x'] - graph.nodes[edge[1]]['x']) ** 2 +
                        (graph.nodes[edge[0]]['y'] - graph.nodes[edge[1]]['y']) ** 2) ** 0.5
            adj_matrix[i][j] = distance
            adj_matrix[j][i] = distance

        # for row in adj_matrix:
        #   print(row)
        feature_matrix1 = process_features_cuda(file_name)

print("Time Taken For GPU is " , timer()-start)
# for row in feature_matrix1:
#   print(row)


Streaming output truncated to the last 5000 lines.
   2.7739494   2.774002    2.7740366   2.8872244   3.203106    3.4905007
   3.669617    3.6696262   4.4586024   4.870987    4.8710437   9.095259
   9.742067   10.318833   11.313378   12.417201  ]
 [ 0.          0.8008299   1.3869611   1.6015301   2.118649    2.1186824
   2.4023304   2.8872468   2.8872743   3.4905257   3.4905326   3.66966
   4.00393     4.1609974   4.237379    4.45857     5.548004    8.474798
   9.095263   10.098947   10.635077   11.679611  ]
 [ 0.          0.8007002   0.8007424   0.8008299   1.3869356   1.6015005
   2.1186764   2.4023142   2.7739542   2.8872266   3.2031002   3.4905012
   3.490581    4.0038924   4.003952    5.0009675   5.6055045   8.360528
   9.024485   10.61999    10.870136   11.794798  ]
 [ 0.          0.8007424   0.80079246  0.8008003   1.3869611   1.3870224
   2.1186967   2.118713    2.7740362   2.887249    2.8872771   2.8873236
   3.20315     3.4905012   4.8046947   5.000888    5.6054754   9.024489